### Download fastText for command line

In [236]:
#! wget https://github.com/facebookresearch/fastText/archive/v0.2.0.zip
#! unzip v0.2.0.zip
#! cd fastText-0.2.0
#! make

### Download benchmark datasets

In [230]:
# Download one of the benchmark dataset
ag_df = pd.read_csv('https://github.com/mhjabreel/CharCnn_Keras/raw/master/data/ag_news_csv/train.csv',header=None
                    ,names=['y','title','text'])
ag_df_test = pd.read_csv('https://github.com/mhjabreel/CharCnn_Keras/raw/master/data/ag_news_csv/test.csv',header=None
                    ,names=['y','title','text'])

In [ ]:
# import IMDB
##IMDB DATA
# getting the imdb review data.
import requests
response = requests.get('http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz',stream=True)
with open('aclImdb_v1.tar.gz','wb') as f:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)
            #f.flush() # forces it to write on disk 


import tarfile
with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as tar:
    tar.extractall()

import os,pandas as pd
import tqdm
#from tqdm import tqdm_notebook as tqdm

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
#pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
with tqdm.tqdm(total=50000) as t:
    for s in ('test', 'train'):
        for l in ('pos', 'neg'):
            path = os.path.join(basepath, s, l)
            for file in os.listdir(path):
                with open(os.path.join(path, file),'r', encoding='utf-8') as infile:
                    txt = infile.read()
                    df = df.append([[txt, labels[l],s]],ignore_index=True)
                    #pbar.update()
                    t.update()
df.columns = ['review', 'sentiment','set']
import numpy as np
np.random.seed(0)
imdb_df = df.reindex(np.random.permutation(df.index))
imdb_df.to_csv('imdb_sentiment.csv', encoding='utf-8')


In [ ]:
## Load IMDB: One of the benchmark from the Baselines and Bigrams
import pandas as pd
imdb_df = pd.read_csv('imdb_sentiment.csv', encoding='utf-8')


In [ ]:
## Movie Review example: Benchmark from Kim et al. 2014: Convolutional Neural Networks for sentence classification
import requests
neg = 'https://raw.githubusercontent.com/yoonkim/CNN_sentence/master/rt-polarity.neg'
pos = 'https://raw.githubusercontent.com/yoonkim/CNN_sentence/master/rt-polarity.pos'
neg = requests.get(neg).text.split('\n')
pos = requests.get(pos).text.split('\n')
docs = neg+pos
# Make train test split
idx = np.random.permutation(np.arange(len(tokenized)))

n = int(len(idx)*0.75)
train,test = idx[0:n],idx[n:]
import pandas as pd
import numpy as np
y = np.array(([0]*len(neg))+([1]*len(pos)))

df = pd.DataFrame({'text':docs,'y':y})
train_df,test_df = df.iloc[train],df.iloc[test]

### Format data for fastText

In [238]:
import nltk
def make_fasttext_format(df,y_col,text_col,outfile,tokenizer=nltk.tokenize.TweetTokenizer().tokenize):
    docs = df[text_col].values
    # tokenize
    tokenized = [' '.join(tokenizer(doc)) for doc in docs]
    # lower
    tokenized = [doc.lower().replace('\n',' __newline__ ') for doc in tokenized]
    fasttext_labels = ['__label__%s'%str(i) for i in df[y_col]]
    fast_docs = [' '.join([fasttext_labels[i],tokenized[i]]) for i in range(len(df))]
    with open(outfile,'w') as f:
        f.write('\n'.join(fast_docs))
    f.close()

In [239]:
make_fasttext_format(ag_df,
                     y_col='y',
                     text_col='text'
                     ,outfile='ag_train')
make_fasttext_format(ag_df_test,
                     y_col='y',
                     text_col='text'
                     ,outfile='ag_test')

make_fasttext_format(imdb_df[imdb_df['set']=='train'],
                     y_col='sentiment',
                     text_col='review'
                     ,outfile='imdb_train')
make_fasttext_format(imdb_df[imdb_df['set']=='test'],
                     y_col='sentiment',
                     text_col='review'
                     ,outfile='imdb_test')
make_fasttext_format(train_df,
                     y_col='y',
                     text_col='text'
                     ,outfile='mr_train')
make_fasttext_format(test_df,
                     y_col='y',
                     text_col='text'
                     ,outfile='mr_test')

## Run fastText classification

In [240]:
# set path to fasttext
fast_path = '/mnt/b0c8e396-e5ba-4614-be6f-146c4c861ab3/fastText-0.2.0'
! {fast_path}/./fasttext supervised -input imdb_train -output model_fast -lr 0.5 -epoch 50 -wordNgrams 3 -dim 10 -ws 5
#! {fast_path}/./fasttext supervised -input fasttext.train -output model_fast -lr 0.5 -epoch 200 -wordNgrams 3 -dim 100 -ws 5
! {fast_path}/./fasttext test model_fast.bin imdb_test

Read 7M words
Number of words:  103093
Number of labels: 2
Progress: 100.0% words/sec/thread: 1269315 lr:  0.000000 loss:  0.030434 ETA:   0h 0m
N	25000
P@1	0.9
R@1	0.9


In [241]:
### Ag example
# set path to fasttext
fast_path = '/mnt/b0c8e396-e5ba-4614-be6f-146c4c861ab3/fastText-0.2.0'
! {fast_path}/./fasttext supervised -input ag_train -output model_fast -lr 0.5 -epoch 50 -wordNgrams 3 -dim 10 -ws 5
#! {fast_path}/./fasttext supervised -input fasttext.train -output model_fast -lr 0.5 -epoch 200 -wordNgrams 3 -dim 100 -ws 5
! {fast_path}/./fasttext test model_fast.bin ag_test

Read 4M words
Number of words:  78902
Number of labels: 4
Progress: 100.0% words/sec/thread: 1224338 lr:  0.000000 loss:  0.031434 ETA:   0h 0m
N	7600
P@1	0.912
R@1	0.912


In [242]:
# set path to fasttext
fast_path = '/mnt/b0c8e396-e5ba-4614-be6f-146c4c861ab3/fastText-0.2.0'
! {fast_path}/./fasttext supervised -input mr_train -output model_fast -lr 0.5 -epoch 300 -wordNgrams 2 -dim 100 -ws 5
#! {fast_path}/./fasttext supervised -input fasttext.train -output model_fast -lr 0.5 -epoch 200 -wordNgrams 3 -dim 100 -ws 5
! {fast_path}/./fasttext test model_fast.bin mr_test

Read 0M words
Number of words:  17808
Number of labels: 2
Progress: 100.0% words/sec/thread:  987497 lr:  0.000000 loss:  0.022001 ETA:   0h 0m
N	2666
P@1	0.761
R@1	0.761


## Baselines and Bigrams implementation

In [ ]:
## Download script for searching the hyperparameters with bayesian optimization using the hyperopt package.
import requests
with open('get_bow_baseline.py','w') as f:
    f.write(requests.get('https://raw.githubusercontent.com/snorreralund/test_tokenization/master/get_bow_baseline.py'))
f.close()

In [6]:
import get_bow_baseline as baseline
baseline_mr = baseline.TokenizationTest(train_df,test_df,MAX_EVALS=150,scoring_function='accuracy_score')
import warnings; warnings.simplefilter('ignore')
import nltk
tokenizer = nltk.tokenize.TweetTokenizer().tokenize
baseline_mr.evaluate('nltk_tweet',tokenizer)

100%|██████████| 150/150 [06:07<00:00,  1.58s/it, best loss: 0.23339999999999994]
Final accuracy and roc_auc score of tokenizer (nltk_tweet) + nb_log: 0.763 and 0.844


In [247]:
imdb_df = imdb_df.drop('Unnamed: 0',axis=1)
imdb_df.columns = ['text','y','set']
imdb_df.head()

,text,y,set
0,Anything Park Chan-wook creates is guaranteed ...,1,test
1,... So some people might argue that this can't...,0,test
2,We are not in the fairy tale of the naked empe...,0,train
3,I saw this show about 3-4 years ago. It was da...,1,train
4,"You know Jason, you know Freddy, and you know ...",0,train


In [253]:
import get_bow_baseline as baseline
baseline_imdb = baseline.TokenizationTest(imdb_df[imdb_df['set']=='train'],imdb_df[imdb_df['set']=='test']
                                          ,MAX_EVALS=100,scoring_function='accuracy_score')
import warnings; warnings.simplefilter('ignore')
import nltk
tokenizer = nltk.tokenize.TweetTokenizer().tokenize
baseline_imdb.evaluate('nltk_tweet',tokenizer)

100%|██████████| 100/100 [3:57:23<00:00, 116.67s/it, best loss: 0.09654400000000007] 
Final accuracy and roc_auc score of tokenizer (nltk_tweet) + nb_log: 0.890 and 0.953
